# **Configurando as bibliotecas do projeto**

## *Instalação de bibliotecas*

## *Importação de bibliotecas*

In [ ]:
'''
    Essa célula será usada para a importação de bibliotecas.
'''

# Instala a biblioteca que será utilizada para se obter os dados financeiros.
import yfinance as yf
# Importa a biblioteca que será utilizada para a manipulação de DataFrames.
import pandas as pd
# Importa a biblioteca que será utilizada para lidar com objetos do tipo "datetime".
from datetime import datetime, timedelta
# Importa a biblioteca que será responsável pela tipagem de funções.
from typing import Union, Optional
# Importa a biblioteca que será utilizada para a manipulação de arrays e também para o uso de algumas funções matemáticas.
import numpy as np

# **Data Engineering**

## *Obtenção dos tickers + Configurações iniciais*

In [ ]:
'''
    Essa célula será usada para se obter os tickets das companhias que fazem parte do S&P 500, com base em dados da wikipedia.
'''

# Salva em uma variável a url que contém a tabela com os tickets das companhias.
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Lê todas as tabelas presentes na url acima.
sp500_table = pd.read_html(url)

# Salva a coluna "Symbol" da primeira tabela em uma variável (tal coluna contém todos os tickets das companhias que fazem parte do S&P 500).
sp500_tickets = sp500_table[0]['Symbol']

# Transforma os tickets obtidos em uma lista.
sp500_tickets = sp500_tickets.tolist()

# Exibe a lista de tickets criada acima.
sp500_tickets

In [ ]:
'''
    Essa célula será usada para definir o intervalo de tempo dos dados que serão coletados, especificando a data de início e a data de fim.
'''

# Define a data inicial cujos dados serão coletados.
start_date = datetime(2018,1,1).date()
# Define a data final cujos dados serão coletados.
end_date = datetime(2024,1,1).date()

In [ ]:
'''
    Essa célula será usada para criar um dicionário chamado setup, que contém os parâmetros principais para a coleta de dados, 
    incluindo a lista de tickers das companhias do S&P 500 e o intervalo de tempo.
'''

# Cria um dicionário para guardar alguns parâmetros importantes para a coleta de dados.
setup = {
    "tickets": sp500_tickets,
    "start_date": start_date,
    "end_date": end_date
}

## *Obtendo alguns dados via yfinance (Open, High, Low, Close, Adj Close, Volume)*

In [ ]:
# Vamos usar os 10 tickers que estão presentes na variável de configuração setup.

# Cria uma variável que guardará a coluna "Adj Close" dos dados referentes a cada um dos tickers presentes na variável de configuração setup.
data_list = []

# Vamos reimportar os dados de cada um dos tickers. Perceba que estamos interessados apenas na coluna "Adj Close" dos dados importados.
for ticker in sp500_tickets[len(sp500_tickets)-10:]:
    # 
    ticker_data = yf.download(ticker, setup['start_date'], setup['end_date'])
    # Adiciono um nome a série temporal obtida acima.
    ticker_data.name=ticker
    # Salvo a série temporal em questão na variável "data_list".
    data_list.append(ticker_data)

## *Feature Engineering*

### *Gerando os retornos aritméticos (diários, semanais e mensais) dos tickers*

In [ ]:
def get_ticker_daily_arithmetic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos aritméticos diários de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_daily_arithmetic_returns (pd.Series): Série temporal dos retornos aritméticos diários do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Cria uma série temporal contendo os retornos diários do ticker em questão.
    ticker_daily_arithmetic_returns = daily_adjusted_closing_prices.pct_change()

    # Como não é possível calcular o retorno diário do "dia 0", removeremos a primeira linha da série temporal gerada acima.
    ticker_daily_arithmetic_returns = ticker_daily_arithmetic_returns[1:]
    
    # Retorna a série temporal resultante.
    return ticker_daily_arithmetic_returns

In [ ]:
def get_ticker_weekly_arithmetic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos aritméticos semanais de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_weekly_arithmetic_returns (pd.Series): Série temporal dos retornos aritméticos semanais do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Transforma a série temporal dos preços de fechamento diários na série temporal dos preços de fechamento semanais.
    weekly_adjusted_closing_prices = daily_adjusted_closing_prices.resample('W').last()
    
    # Cria uma série temporal contendo os retornos semanais do ticker em questão.
    ticker_weekly_arithmetic_returns = weekly_adjusted_closing_prices.pct_change()

    # Como não é possível calcular o retorno semanal da "semana 0", removeremos a primeira linha da série temporal gerada acima.
    ticker_weekly_arithmetic_returns = ticker_weekly_arithmetic_returns[1:]
    
    # Retorna a série temporal resultante.
    return ticker_weekly_arithmetic_returns

In [ ]:
def get_ticker_monthly_arithmetic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos aritméticos mensais de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_monthly_arithmetic_returns (pd.Series): Série temporal dos retornos aritméticos mensais do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Transforma a série temporal dos preços de fechamento diários na série temporal dos preços de fechamento mensais.
    monthly_adjusted_closing_prices = daily_adjusted_closing_prices.resample('M').last()
    
    # Cria uma série temporal contendo os retornos mensais do ticker em questão.
    ticker_monthly_arithmetic_returns = monthly_adjusted_closing_prices.pct_change()

    # Como não é possível calcular o retorno mensal no "mês 0", removeremos a primeira linha da série temporal gerada acima.
    ticker_monthly_arithmetic_returns = ticker_monthly_arithmetic_returns[1:]
    
    # Retorna a série temporal resultante.
    return ticker_monthly_arithmetic_returns

### *Gerando os retornos logarítmicos (diários, semanais e mensais) dos tickers*

In [ ]:
def get_ticker_daily_logarithmic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos logarítmicos diários de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_daily_arithmetic_returns (pd.Series): Série temporal dos retornos logarítmicos diários do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Obtem a série temporal dos retornos diários do ticker em questão
    ticker_daily_arithmetic_returns = get_ticker_daily_arithmetic_returns(daily_adjusted_closing_prices)
    
    # Transforma a série temporal dos retornos diários do ticker em questão em sua versão logarítmica.
    ticker_daily_logarithmic_returns = ticker_daily_arithmetic_returns.apply(lambda x: np.log(1 + x))
    
    # Retorna a série temporal resultante.
    return ticker_daily_logarithmic_returns

In [ ]:
def get_ticker_weekly_logarithmic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos logarítmicos semanais de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_weekly_logarithmic_returns (pd.Series): Série temporal dos retornos logarítmicos semanais do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Obtem a série temporal dos retornos semanais do ticker em questão
    ticker_weekly_arithmetic_returns = get_ticker_weekly_arithmetic_returns(daily_adjusted_closing_prices)
    
    # Transforma a série temporal dos retornos semanais do ticker em questão em sua versão logarítmica.
    ticker_weekly_logarithmic_returns = ticker_weekly_arithmetic_returns.apply(lambda x: np.log(1 + x))
    
    # Retorna a série temporal resultante.
    return ticker_weekly_logarithmic_returns

In [ ]:
def get_ticker_monthly_logarithmic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos logarítmicos mensais de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_weekly_logarithmic_returns (pd.Series): Série temporal dos retornos logarítmicos mensais do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Obtem a série temporal dos retornos semanais do ticker em questão
    ticker_monthly_arithmetic_returns = get_ticker_monthly_arithmetic_returns(daily_adjusted_closing_prices)
    
    # Transforma a série temporal dos retornos semanais do ticker em questão em sua versão logarítmica.
    ticker_monthly_logarithmic_returns = ticker_monthly_arithmetic_returns.apply(lambda x: np.log(1 + x))
    
    # Retorna a série temporal resultante.
    return ticker_monthly_logarithmic_returns

### *Calculando os retornos cumulativos logarítmicos (diário, semanal e mensal) dos tickers*

In [ ]:
'''
    Não julgo que seja necessário criar uma função para se calcular os retornos logarítmicos, visto que, basta que usemos o método 'cumsum()'
    no retorno das funções "get_ticker_daily_logarithmic_returns", "get_ticker_weekly_logarithmic_returns" e 
    "get_ticker_monthly_logarithmic_returns" para que obtenhamos os retornos cumulativos desejados.
'''

## *Obtendo algumas métricas*

### *Métricas de volatilidade*

*Ainda não sei bem quais medidas calcular aqui, visto que eu ainda não sei quais tipos de volatilidade calcular (volatilidades escalares ou volatilidades vetoriais)*

In [135]:
def get_ticker_daily_historical_volatility(daily_adjusted_closing_prices: pd.Series) -> float:
    '''
        Description:
            Essa função calcula a volatilidade histórica diária de um certo ticker baseada nos retornos diários logarítmicos deste.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_daily_historical_volatility (float): Número que representa a volatilidade histórica diária do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''
    
    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Calcula a volatilidade histórica diária do ticker em questão.
    ticker_daily_historical_volatility = get_ticker_daily_logarithmic_returns(daily_adjusted_closing_prices).std()
    
    # Retorna o resultado obtido.
    return ticker_daily_historical_volatility

In [143]:
def get_ticker_weekly_historical_volatility(daily_adjusted_closing_prices: pd.Series) -> float:
    '''
        Description:
            Essa função calcula a volatilidade histórica semanal de um certo ticker baseada nos retornos semanais logarítmicos deste.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_weekly_historical_volatility (float): Número que representa a volatilidade histórica semanal do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''
    
    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Calcula a volatilidade histórica semanal do ticker em questão.
    ticker_weekly_historical_volatility = get_ticker_weekly_logarithmic_returns(daily_adjusted_closing_prices).std()
    
    # Retorna o resultado obtido.
    return ticker_weekly_historical_volatility

In [146]:
def get_ticker_monthly_historical_volatility(daily_adjusted_closing_prices: pd.Series) -> float:
    '''
        Description:
            Essa função calcula a volatilidade histórica mensal de um certo ticker baseada nos retornos mensais logarítmicos deste.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_monthly_historical_volatility (float): Número que representa a volatilidade histórica mensal do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''
    
    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Calcula a volatilidade histórica mensal do ticker em questão.
    ticker_monthly_historical_volatility = get_ticker_monthly_logarithmic_returns(daily_adjusted_closing_prices).std()
    
    # Retorna o resultado obtido.
    return ticker_monthly_historical_volatility

In [132]:
def get_ticker_moving_volatility(daily_adjusted_closing_prices: pd.Series, time_interval: int) -> pd.Series:
    '''
        Description:
            Essa função calcula a volatilidade móvel de "time_interval" dias de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
            time_interval (int): Quantidade de dias da janela móvel.
        Return:
            moving_volatility (pd.Series): Série temporal que representa a evolução da volatilidade móvel de "time_interval" dias do ticker
                                           em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            TypeError: É esperado que o parâmetro "time_interval" seja do tipo int.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''
    
    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso o parâmetro 'time_interval' não seja do tipo int, um erro é retornado.
    if not isinstance(time_interval, int):
        raise TypeError("É esperado que o parâmetro 'time_interval' seja do tipo int.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Calcula a volatilidade móvel de "time_interval" dias da série temporal "daily_adjusted_closing_prices".
    moving_volatility = daily_adjusted_closing_prices.rolling(window=time_interval).std()
    
    # Remove os primeiros (time_interval - 1) dados da série temporal gerada acima, isto é, remove os dados NaN de tal série temporal.
    moving_volatility = moving_volatility[time_interval-1:]

    # Retorna o resultado obtido.
    return moving_volatility

### *Métricas de tendência e momentum*

*Médias móveis simples, médias móveis exponenciais e índice de força relativa*

In [ ]:
def get_ticker_simple_moving_average(daily_adjusted_closing_prices: pd.Series, time_interval: int) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal que representa a média móvel simples de "time_interval" dias de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
            time_interval (int): Quantidade de dias da janela móvel.
        Return:
            ticker_simple_moving_average (pd.Series): Série temporal que representa a média móvel simples de "time_interval" dias do ticker
            em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            TypeError: É esperado que o parâmetro "time_interval" seja do tipo int.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso o parâmetro 'time_interval' não seja do tipo int, um erro é retornado.
    if not isinstance(time_interval, int):
        raise TypeError("É esperado que o parâmetro 'time_interval' seja do tipo int.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Calcula e salva a série temporal que representa a média móvel simples de "time_interval" dias do ticker em questão.
    ticker_simple_moving_average = daily_adjusted_closing_prices.rolling(window=time_interval).mean()
    
    # Remove os valores nulos gerados na criação da série temporal acima.
    ticker_simple_moving_average = ticker_simple_moving_average.dropna()
    
    # Retorna a série temporal resultante.
    return ticker_simple_moving_average

In [ ]:
def get_ticker_exponential_moving_average(daily_adjusted_closing_prices: pd.Series, time_interval: int, 
                                          remove_adjustment_period: Optional[bool]  = True) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal que representa a média móvel exponencial de "time_interval" dias de um certo ticker.
            Tal média é calculada de forma não ajustada, isto é, a média é calculada recursivamente sem ajuste dos pesos, resultando em uma 
            média que responde mais rapidamente às mudanças recentes.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
            time_interval (int): Quantidade de dias da janela móvel.
            remove_adjustment_period (bool) [Default = True]: Se True, remove os primeiros "time_interval" dias da série temporal, isto é, 
                                                              remove o período de ajuste dos dados. Caso contrário, mantém esse 
                                                              período na série temporal.
        Return:
            ticker_exponential_moving_average (pd.Series): Série temporal que representa a média móvel exponencial de "time_interval" 
            dias do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            TypeError: É esperado que o parâmetro "time_interval" seja do tipo int.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso o parâmetro 'time_interval' não seja do tipo int, um erro é retornado.
    if not isinstance(time_interval, int):
        raise TypeError("É esperado que o parâmetro 'time_interval' seja do tipo int.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Calcula e salva a série temporal que representa a média móvel exponencial de "time_interval" dias do ticker em questão.
    ticker_exponential_moving_average = daily_adjusted_closing_prices.ewm(span=time_interval, adjust=False).mean()
    
    # Caso remove_adjustment_period seja igual à True, os primeiros "time_interval" dias serão removidos da série temporal gerada acima.
    if remove_adjustment_period: 
        ticker_exponential_moving_average = ticker_exponential_moving_average[time_interval-1:]
        
    # Retorna a série temporal resultante.
    return ticker_exponential_moving_average

In [110]:
def calculate_rsi_with_sma(daily_adjusted_closing_prices: pd.Series, time_interval: int) -> pd.Series:
    '''
        Description:
            Essa função calcula o Índice de Força Relativa (RSI) usando médias móveis simples (SMA) para a série temporal de preços
            "daily_adjusted_closing_prices" que representa os preços de fechamento diários ajustados de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
            time_interval (int): Quantidade de dias da janela móvel.
        Return:
            rsi (pd.Series): Série temporal que representa a evolução do índice de força relativa da série temporal 
                             "daily_adjusted_closing_prices" calculado através de médias móveis simples.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            TypeError: É esperado que o parâmetro "time_interval" seja do tipo int.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''
    
    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso o parâmetro 'time_interval' não seja do tipo int, um erro é retornado.
    if not isinstance(time_interval, int):
        raise TypeError("É esperado que o parâmetro 'time_interval' seja do tipo int.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Calcula a variação diária de preços da série temporal "daily_adjusted_closing_prices".
    price_variations = daily_adjusted_closing_prices.diff()
    
    # Separa os ganhos das perdas filtrando todos os price_variations maiores que zero e setando como zero o valor de todos os price_variations
    # menores que 0.
    gains = price_variations.where(price_variations > 0, 0)
    # Separa as perdas dos ganhos filtrando todos os price_variations menores que zero e setando como zero o valor de todos os price_variations
    # maiores que 0.
    losses = -price_variations.where(price_variations < 0, 0)
    
    # Calcula as médias móveis simples de "time_interval" dias dos ganhos e das perdas
    average_gain = gains.rolling(window=time_interval).mean()
    average_loss = losses.rolling(window=time_interval).mean()
    
    # Calcula a força relativa (RS)
    rs = average_gain/average_loss
    
    # Calcula o RSI
    rsi = 100 - (100 / (1 + rs))
    
    # Remove os primeiros (time_interval - 1) valores da série temporal "rsi" (que são valores NaN) e retorna o resultado obtido.
    return rsi[time_interval-1:]

In [115]:
def calculate_rsi_with_ema(daily_adjusted_closing_prices: pd.Series, time_interval: int,
                           remove_adjustment_period: Optional[bool]  = True) -> pd.Series:
    '''
        Description:
            Essa função calcula o Índice de Força Relativa (RSI) usando médias móveis exponenciais (EMA) para a série temporal de preços
            "daily_adjusted_closing_prices" que representa os preços de fechamento diários ajustados de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
            time_interval (int): Quantidade de dias da janela móvel.
            remove_adjustment_period (bool) [Default = True]: Se True, remove os primeiros "time_interval" dias da série temporal, isto é, 
                                                              remove o período de ajuste dos dados. Caso contrário, mantém esse 
                                                              período na série temporal.
        Return:
            rsi (pd.Series): Série temporal que representa a evolução do índice de força relativa da série temporal 
                             "daily_adjusted_closing_prices" calculado através de médias móveis exponenciais.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            TypeError: É esperado que o parâmetro "time_interval" seja do tipo int.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''
    
    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso o parâmetro 'time_interval' não seja do tipo int, um erro é retornado.
    if not isinstance(time_interval, int):
        raise TypeError("É esperado que o parâmetro 'time_interval' seja do tipo int.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Calcula a variação diária de preços da série temporal "daily_adjusted_closing_prices".
    price_variations = daily_adjusted_closing_prices.diff()
    
    # Separa os ganhos das perdas filtrando todos os price_variations maiores que zero e setando como zero o valor de todos os price_variations
    # menores que 0.
    gains = price_variations.where(price_variations > 0, 0)
    # Separa as perdas dos ganhos filtrando todos os price_variations menores que zero e setando como zero o valor de todos os price_variations
    # maiores que 0.
    losses = -price_variations.where(price_variations < 0, 0)
    
    # Calcula as médias móveis exponenciais de "time_interval" dias dos ganhos e das perdas.
    average_gain = gains.ewm(span=time_interval, adjust=False).mean()
    average_loss = losses.ewm(span=time_interval, adjust=False).mean()
    
    # Calcula a força relativa (RS).
    rs = average_gain/average_loss
    
    # Calcula o RSI.
    rsi = 100 - (100 / (1 + rs))
    
    # Caso remove_adjustment_period seja igual à True, os primeiros "time_interval" dias serão removidos da série temporal gerada acima.
    if remove_adjustment_period: 
        rsi = rsi[time_interval-1:]
    
    # Retorna o resultado obtido.
    return rsi

### *Métricas de reversão de média*

*Bandas de bollinger + (Pensar em uma métrica como o Z-score dos retornos, mas que não assuma normalidade)*

## *Limpando os dados*

# **Machine Learning**

# **Ideias**

Creio que nesse projeto teremos que usar um banco de dados SQL e um banco de dados NOSQL.

A ideia por trás de usar um banco de dados SQL é que, nesse banco de dados, sejam salvos os seguintes dados:
- Preços históricos de abertura    
-  Preços históricos de fechamento
- Preços máximos diários
- Preços mínimos diários
- Preços de fechamento ajustados
- Volume negociado no dia (Talvez seja interessante criar novas features a partir dessa)
- Retornos aritméticos (Diários)
- Retornos logarítmicos (Diários)
- Retornos cumulativos (Diários)
  
Já em um banco de dados NOSQL, a ideia seria salvar os seguintes dados:
- Retornos aritméticos (Semanais / Mensais)
- Retornos logarítmicos (Semanais / Mensais)
- Retornos cumulativos (Semanais / Mensais)
- Volatilidade histórica + Outras medidas de volatilidade
- Métricas de tendência e momentum
- Métricas de reversão de média
- Resultados das análises de sentimento (Ainda preciso pensar sobre)
  
O principal motivo para que sejam usados dois bancos de dados é que, em um banco de dados SQL o formato das tabelas é rigido, logo não sei se seria possível criar um design de banco de dados que salve todas essas informações para cada ticket, já que, existem informações aqui que são escalares e existem, além disso, diferentes informações vetoriais de tamanhos distintos. Creio que um banco de dados NOSQL resolva essa situação.

# **A fazer**

- Conversar com o professor sobre a questão das métricas de volatilidade
- Criar o desing do banco de dados SQL
- Pensar no design do banco de dados NOSQL
- Conversar com o professor sobre o LangChain e decidir como essa parte será feita.